<h1>
    <font color='green'>
                        Processo de extração de dados da web
     </font>
    
</h1>

Obtendo dados da CVM.<br/> 
Calcular o beta dos fundos.<br/>
1. [Informações](#1)<br>
a. [Versionamento](#2)<br>
b. [Funções usadas](#3)<br>
2. [Processo](#4)<br>
a. [Bibliotecas](#5) <br>
b. [Encontra arquivos que serão importados](#6)<br>
c. [Momento da importação](#7)<br>
d. [Tratamento de dados](#8)<br>
e. [Exclui dados desnecessários](#9)<br>
f. [Analise de um fundo](#10)<br>
g. [](#11)<br>


<a id =2></a><br>
### Versionamento

| Versão | Descrição | Data | Desenvolvedor
| :- |:------------- | :-: | :-:
|v1.0.0|Versão inicial|2021-05-10|Marco Cruz
|v1.0.1|Processo de ETL|2021-06-06|Marco Cruz
|v1.0.2|Ajustes|2021-06-23|Marco Cruz

<a id =5></a><br>
### Bibliotecas

In [2]:
#
# definição das bibliotecas necessárias
#
#import requests
#from bs4 import BeautifulSoup
import sqlite3 as sql
from pathlib import Path
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.2f' % x)
import matplotlib.pyplot as plt
#from datetime import datetime
import numpy as np
import aux_funcoes as fa
import aux_consultas_sql as csql
import aux_listas as la
#print('-' * 75)
print('-- definição das bibliotecas necessárias')
print('-' * 75)

-- definição das bibliotecas necessárias
---------------------------------------------------------------------------


<a id =6></a><br>
### Encontra arquivos que serão importados

<div class="alert-warning">    
    <h4><br>&nbsp; listando arquivos disponíveis para carga no site da CVM...<br>
    </h4>
</div>

In [3]:
#
# define arquivos a serem importados
#
# Portal Dados Abertos CVM


print('-- define arquivos a serem importados')
print('-' * 75)

url_cotas = 'http://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/'
url_carac = 'http://dados.cvm.gov.br/dados/FI/DOC/EXTRATO/DADOS/'
ext = 'csv'
lst_arq_cotas = fa.get_url_paths(url_cotas, ext)
print('-- - Quantidade de arquivos no diretório de cotas: ' +str(len(lst_arq_cotas)))
lst_arq_carac = fa.get_url_paths(url_carac, ext)
print('-- - Quantidade de arquivos no diretório de características: ' +str(len(lst_arq_carac)))
print('-' * 75)

-- define arquivos a serem importados
---------------------------------------------------------------------------
-- - Quantidade de arquivos no diretório de cotas: 54
-- - Quantidade de arquivos no diretório de características: 8
---------------------------------------------------------------------------


### Retira da tabela de cotas os dois meses mais recentes

<div class="alert-warning">    
    <h4><br>&nbsp; Mantendo apenas últimos meses na fila de importação...<br>
    </h4>
</div>

In [4]:
from datetime import datetime
today = datetime.today()
datem0 = datetime(today.year, today.month, 1).strftime("%Y%m")
datem1 = datetime(today.year, today.month-1, 1).strftime("%Y%m")
datem2 = datetime(today.year, today.month-2, 1).strftime("%Y%m")
lst_meses = (datem0,datem1,datem2)


lst_arq_cotas = fa.filtra_lista(lst_arq_cotas,lst_meses)
print(lst_arq_cotas)

['http://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_202104.csv', 'http://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_202105.csv', 'http://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_202106.csv']


<div class="alert-warning">    
    <h4><br>&nbsp; Excluindo dados recentes na tablea de cotações...<br>
    </h4>
</div>

In [5]:
print('--  - data e hora inicio = ', fa.agora())
conn = sql.connect('investdb.db')
with conn:
    cur = conn.cursor()
    cur.execute(csql.sqlDelCotas())
    res = cur.fetchall()
    print(res)
if conn:
    conn.close()

print('--  - data e hora final  = ', fa.agora())


--  - data e hora inicio =  27/06/2021 21:04:28
[]
--  - data e hora final  =  27/06/2021 21:04:31


<a id =7> </a><br>
### Momento da importação

<div class="alert-warning">    
    <h4><br>&nbsp; importando cotações para o dataframe <u>df</u>...<br>
    </h4>
</div>

In [6]:
print('-' * 75)
print('-- Arquivos disponíveis p importação: ' + str(len(lst_arq_cotas)))
print('-- Inicio da importação dos csvs (cotas)')
print('--  - data e hora inicio = ', fa.agora())
df = (pd.read_csv(f, sep=';') for f in lst_arq_cotas)
df = pd.concat(df)
print('-- Fim da importação dos csvs')
print('--  - data e hora final  = ', fa.agora())
print('-' * 75)
print('-- formato')
print(df.shape)
print('-' * 75)
#print('-- cabeçalho')
#print(df.head(3))
#print('-' * 75)

---------------------------------------------------------------------------
-- Arquivos disponíveis p importação: 3
-- Inicio da importação dos csvs (cotas)
--  - data e hora inicio =  27/06/2021 21:04:58
-- Fim da importação dos csvs
--  - data e hora final  =  27/06/2021 21:07:07
---------------------------------------------------------------------------
-- formato
(1196642, 9)
---------------------------------------------------------------------------


<div class="alert-warning">    
    <h4><br>&nbsp; importando caracteristicas para o dataframe <u>df2</u>...<br>
    </h4>
</div>

In [7]:
print('-- Inicio da importação dos csvs (caracteristicas)')
print('--  - data e hora inicio = ', fa.agora())
df2 = (pd.read_csv(f, sep=';', encoding = 'cp1252') for f in lst_arq_carac)
df2 = pd.concat(df2)
print('-- Fim da importação dos csvs')
print('--  - data e hora final = ', fa.agora())
print('-- formato')
print(df2.shape)


-- Inicio da importação dos csvs (caracteristicas)
--  - data e hora inicio =  27/06/2021 21:07:07


<ipython-input-7-8af5f0076ad6>:4: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  df2 = pd.concat(df2)


-- Fim da importação dos csvs
--  - data e hora final =  27/06/2021 21:08:12
-- formato
(73645, 116)


<div class="alert-warning">    
    <h4><br>&nbsp; importando selic para o dataframe <u>selic</u>...<br>
    </h4>
</div>

In [9]:
# https://www.bcb.gov.br/htms/SELIC/SELICdiarios.asp?frame=1
selic = pd.read_json('http://api.bcb.gov.br/dados/serie/bcdata.sgs.{}/dados?formato=json'.format(11))
selic['data'] = pd.to_datetime(selic['data'], format = '%d/%m/%Y')
selic['valor'] = selic['valor']/100 
#calculates decimal rate from the percentual value
selic.rename(columns = {'data':'DT_COTACAO', 'valor':'TAXA'}, inplace = True)
#selic=selic.style.format({'TAXA':'{:10.9f}'})

# cria coluna MES_COMPTC (para auxiliar mais adiante)
selic['MES_COTACAO']=selic['DT_COTACAO'].apply(lambda x : x.replace(day=1))
selic=selic[['DT_COTACAO','TAXA','MES_COTACAO']]


#
# encontra a ultima data de cada mês
#
selic_dt = selic.groupby(['MES_COTACAO']).agg({'DT_COTACAO':'max'})
selic_dt = selic_dt.reset_index()
meses = selic_dt['MES_COTACAO']
selic_dt = selic_dt.drop(['MES_COTACAO'], axis=1)
selic_dt.rename(columns={'DT_COTACAO':'UltimoDiaMes'}, inplace=True)

#
# exclui datas desnecessárias do dataframe
#
print('selic contem:')
print(selic.shape)
print('-'*75)
print('excluindo datas desnecessárias....')
selic= selic[selic['DT_COTACAO'].isin(selic_dt['UltimoDiaMes'])]
print('selic contem:')
print(selic.shape)

selic.to_sql('tbl_selic', conn , index=False) 
print('-'*75)


selic contem:
(8787, 3)
---------------------------------------------------------------------------
excluindo datas desnecessárias....
selic contem:
(421, 3)


ProgrammingError: Cannot operate on a closed database.

<a id =8> </a><br>
### Tratamento de dados

<div class="alert-warning">    
    <h4><br>&nbsp; Tratando datas no dataframe <u>df</u>...<br>
            &nbsp; Criando campo mês competência....<br>
            &nbsp; Limpando caraceteres especiais do CNPJ...        
        <br>
    </h4>
</div>

In [ ]:
# converte coluna DT_COMPTC para tipo data
df['DT_COMPTC'] =  pd.to_datetime(df['DT_COMPTC'], infer_datetime_format=True)

# cria coluna MES_COMPTC (para auxiliar mais adiante)
df['MES_COMPTC']=df['DT_COMPTC'].apply(lambda x : x.replace(day=1))

#Trata caracteres especiais no cnpj
df['CNPJ_FUNDO']=df['CNPJ_FUNDO'].str.replace(r'\D', '')

ultimo_pregao = df['DT_COMPTC'].max()
ultimo_pregao

<div class="alert-warning">    
    <h4><br>&nbsp; Última data de cada mês no dataframe <u>df</u>...<br>
    </h4>
</div>

In [50]:
#
# encontra a ultima data de cada mês
#
df_dt = df.groupby(['MES_COMPTC']).agg({'DT_COMPTC':'max'})
df_dt = df_dt.reset_index()
meses = df_dt['MES_COMPTC']
df_dt = df_dt.drop(['MES_COMPTC'], axis=1)
df_dt.rename(columns={'DT_COMPTC':'UltimoDiaMes'}, inplace=True)


<div class="alert-warning">    
    <h4><br>&nbsp; Tratando datas no dataframe <u>df2</u>...<br>
            &nbsp; Criando campo mês competência....<br>
            &nbsp; Limpando caraceteres especiais do CNPJ...        
        <br>
    </h4>
</div>

In [10]:
# converte coluna DT_COMPTC para tipo data
df2['DT_COMPTC'] =  pd.to_datetime(df2['DT_COMPTC'], infer_datetime_format=True)

# cria coluna MES_COMPTC (para auxiliar mais adiante)
df2['MES_COMPTC']=df2['DT_COMPTC'].apply(lambda x : x.replace(day=1))

#Trata caracteres especiais no cnpj
df2['CNPJ_FUNDO']=df2['CNPJ_FUNDO'].str.replace(r'\D', '')

<div class="alert-warning">    
    <h4><br>&nbsp; Última data de cada mês no dataframe <u>df2</u>...<br>
    </h4>
</div>

In [11]:
#
# encontra a ultima data de cada mês
#
df_dt2 = df2.groupby(['MES_COMPTC']).agg({'DT_COMPTC':'max'})
df_dt2 = df_dt2.reset_index()
df_dt2 = df_dt2.drop(['MES_COMPTC'], axis=1)
df_dt2.rename(columns={'DT_COMPTC':'UltimoDiaMes'}, inplace=True)

<a id =9> </a><br>
### Exclui dados desnecessários
<br>posições intermediárias do mês

<div class="alert-warning">    
    <h4><br>&nbsp;
        Exclui colunas desnecessárias dos dataframes...
        <br>
    </h4>
</div>


In [12]:
# drop by Name
df = df.drop(la.lst_campos_excluir, axis=1)
df2 = df2.drop(la.lst_campos_excluir2, axis=1)

<div class="alert-warning">    
    <h4><br>&nbsp;
        Exclui datas desnecessárias do dataframe <u>df</u>...
        <br>
    </h4>
</div>


In [54]:
#
# exclui datas desnecessárias do dataframe
#
print('df contem:')
print(df.shape)
print('-'*75)
print('excluindo datas desnecessárias....')
df= df[df['DT_COMPTC'].isin(df_dt['UltimoDiaMes'])]
print('df contem:')
print(df.shape)
print('-'*75)


df contem:
(1112007, 6)
---------------------------------------------------------------------------
excluindo datas desnecessárias....
df contem:
(61696, 6)
---------------------------------------------------------------------------


<div class="alert-warning">    
    <h4><br>&nbsp;
            Exclui datas desnecessárias do dataframe <u>df2</u>...
        <br>
    </h4>
</div>


In [13]:
#
# exclui datas desnecessárias do dataframe
#
print('df2 contem:')
print(df2.shape)
print('-'*75)
print('excluindo datas desnecessárias....')
df2 = df2[df2['DT_COMPTC'].isin(df_dt2['UltimoDiaMes'])]
print('df2 contem:')
print(df2.shape)
print('-'*75)

#
# refaz o índice
#
df2.sort_values(by=['CNPJ_FUNDO','DT_COMPTC'], inplace=True, ascending=[True, False])
df2 = df2.reset_index(drop=True)

df2 contem:
(73645, 50)
---------------------------------------------------------------------------
excluindo datas desnecessárias....
df2 contem:
(4198, 50)
---------------------------------------------------------------------------


In [56]:
#
# refaz o índice
#

df.sort_values(by=['CNPJ_FUNDO','DT_COMPTC'], inplace=True, ascending=[True, True])
df = df.reset_index(drop=True)

In [57]:
df

,CNPJ_FUNDO,DT_COMPTC,VL_TOTAL,VL_QUOTA,VL_PATRIM_LIQ,MES_COMPTC
0,00017024000153,2021-04-30,1078817.89,27.43,1078492.85,2021-04-01
1,00017024000153,2021-05-31,1072969.15,27.47,1076828.38,2021-05-01
2,00017024000153,2021-06-18,1073659.96,27.49,1074717.51,2021-06-01
3,00068305000135,2021-04-30,41643654.55,27.50,41628745.24,2021-04-01
4,00068305000135,2021-05-31,41121053.09,27.54,41068713.46,2021-05-01
...,...,...,...,...,...,...
61691,97929197000180,2021-05-31,220285167.02,7.06,217142819.69,2021-05-01
61692,97929197000180,2021-06-18,220779921.37,7.08,217834119.10,2021-06-01
61693,97929213000134,2021-04-30,72480985.48,9.54,72470912.77,2021-04-01
61694,97929213000134,2021-05-31,72897731.19,9.59,72886094.48,2021-05-01


<div class="alert-warning">    
    <h4><br>&nbsp;
            VACUUM no SQLite
        <br>
    </h4>
</div>


In [58]:
print('-'*75)
conn = sql.connect('investdb.db')
print(' - Inicio da otimização: ' + fa.agora())
fa.otimizarSQLite(conn)
print(' - Fim da otimização:    ' + fa.agora())
print('-'*75)

---------------------------------------------------------------------------
 - Inicio da otimização: 23/06/2021 00:09:37
 - Fim da otimização:    23/06/2021 00:09:45
---------------------------------------------------------------------------


<div class="alert-warning">    
    <h4><br>&nbsp;
            Agrupando meses e contando registros a serem importados...
        <br>
    </h4>
</div>


In [65]:
# Agrupando...
#df.groupby(['DT_COMPTC']).count()
#df.groupby(['MES_COMPTC']).agg(['mean','count'])

df.groupby(['MES_COMPTC']).size().reset_index(name='QUANTIDADE')

,MES_COMPTC,QUANTIDADE
0,2021-04-01,20525
1,2021-05-01,20811
2,2021-06-01,20360


<div class="alert-warning">    
    <h4><br>&nbsp;
            Carga das cotas atualizadas no SQLite
        <br>
    </h4>
</div>


In [ ]:
#
# faz carga no banco de dados
#
print(' - Inicio da importação do dataframe p o sql: ' + fa.agora())
df.to_sql('tbl_cotas', conn,  schema='investdb',index = False, if_exists='append')
print(' - Fim da importação do dataframe: ' + fa.agora())
print('-'*75)

<div class="alert-warning">    
    <h4><br>&nbsp;
            Carga das caracteristicas atualizadas no SQLite
        <br>
    </h4>
</div>

In [22]:
df2

,CNPJ_FUNDO,DENOM_SOCIAL,DT_COMPTC,CONDOM,PUBLICO_ALVO,CLASSE_ANBIMA,FUNDO_ESPELHO,APLIC_MIN,QT_DIA_PAGTO_RESGATE,TP_DIA_PAGTO_RESGATE,...,PR_ACAO_MAX,PR_DEBENTURE_MIN,PR_DEBENTURE_MAX,PR_DERIV_MIN,PR_DERIV_MAX,PR_COTA_FIP_MIN,PR_COTA_FIP_MAX,PR_COTA_FICFIP_MIN,PR_COTA_FICFIP_MAX,MES_COMPTC
1,00068305000135,FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE IN...,2020-10-30,ABERTO,PÚBLICO EM GERAL,RENDA FIXA LIVRE DURAÇÃO - GRAU DE INVESTIMENTO,S,0.00,0,DIAS ÚTEIS,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2020-10-01
3,00083181000167,OPP I FUNDO DE INVESTIMENTO EM AÇÕES BDR NÍVEL...,2019-09-30,FECHADO,INVESTIDORES PROFISSIONAIS,AÇÕES - FUNDOS FECHADOS,NaN,0.00,3,DIAS ÚTEIS,...,0.00,0.00,33.00,0.00,0.00,0.00,33.00,0.00,33.00,2019-09-01
4,00346750000110,NIC YIELD FUNDO DE INVESTIMENTO EM COTAS DE FU...,2016-06-30,ABERTO,PÚBLICO EM GERAL,MULTIMERCADO - ESTRATÉGIA - JUROS E MOEDAS,N,500.00,1,DIAS ÚTEIS,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2016-06-01
6,00598452000117,MB MASTER REFERENCIADO DI LONGO PRAZO FUNDO DE...,2016-05-31,ABERTO,PÚBLICO EM GERAL,RENDA FIXA BAIXA DURAÇÃO - GRAU DE INVESTIMENTO,NaN,0.00,0,DIAS ÚTEIS,...,0.00,0.00,5.00,0.00,100.00,0.00,10.00,0.00,10.00,2016-05-01
9,00601692000123,FAMA FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS ...,2016-02-29,ABERTO,PÚBLICO EM GERAL,AÇÕES - ATIVO - LIVRE,N,0.00,60,DIAS ÚTEIS,...,100.00,0.00,50.00,0.00,0.00,0.00,0.00,0.00,0.00,2016-02-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4192,96498985000104,BRADESCO FUNDO DE INVESTIMENTO EM COTAS DE FUN...,2016-06-30,ABERTO,PÚBLICO EM GERAL,AÇÕES - INDEXADO - ÍNDICE PASSIVO,N,500.00,0,DIAS ÚTEIS,...,0.00,0.00,0.00,0.00,100.00,0.00,0.00,0.00,0.00,2016-06-01
4193,97519521000191,ITAÚ FLEXPREV ANABELA MULTIMERCADO - FUNDO DE ...,2016-10-31,ABERTO,PREVIDENCIÁRIO,MULTIMERCADO - ESTRATÉGIA - LIVRE,N,1.00,0,DIAS CORRIDOS,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2016-10-01
4195,97519573000168,SERTÃO MULTIMERCADO CRÉDITO PRIVADO - FUNDO DE...,2017-09-29,ABERTO,PÚBLICO EM GERAL,MULTIMERCADO - ESTRATÉGIA - CAPITAL PROTEGIDO,NaN,1000000.00,0,DIAS CORRIDOS,...,100.00,0.00,100.00,0.00,100.00,0.00,100.00,0.00,100.00,2017-09-01
4196,97548164000190,DYNAMIC II - FUNDO DE INVESTIMENTO MULTIMERCAD...,2019-09-30,FECHADO,INVESTIDORES PROFISSIONAIS,MULTIMERCADO - INVESTIMENTO NO EXTERIOR,NaN,0.00,1,DIAS ÚTEIS,...,0.00,0.00,100.00,0.00,0.00,0.00,50.00,0.00,50.00,2019-09-01


In [19]:
#
# faz carga no banco de dados
#
print(' - Inicio da importação do dataframe p o sql: ' + fa.agora())
#df2.to_sql('tbl_caracteristicas', conn,  schema='investdb',index = False, if_exists='append')
print(df2.shape)
df2.drop_duplicates(subset=['CNPJ_FUNDO'], keep='last', inplace=True)

print(df2.shape)
df2.to_csv('tbl_caracteristicas.csv', index=False)
#df2.to_csv('tbl_caracteristicas.csv', encoding='utf-8', index=False)
print(' - Fim da importação do dataframe: ' + fa.agora())
print('-'*75)

 - Inicio da importação do dataframe p o sql: 27/06/2021 21:27:20
(4198, 50)
(4198, 50)
 - Fim da importação do dataframe: 27/06/2021 21:27:20
---------------------------------------------------------------------------


<div class="alert-warning">    
    <h4><br>&nbsp;
            VACUUM no SQLite
        <br>
    </h4>
</div>


In [21]:
df2.drop_duplicates(subset=['CNPJ_FUNDO'], keep='last', inplace=True)

In [ ]:
#
# VACUUM no sqLite
#
print('Inicio da otimização')
fa.agora()
fa.otimizarSQLite(conn)
conn.close()
print('Fim da otimização')
fa.agora()

[Marco Cruz no LinkedIn](https://www.linkedin.com/in/marcoacf/)